In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler

FILE_CLUSTERS = "clusters.txt"
FILE_COORDS = "coordinates.txt"

def load_data(filepath):
    try:
        df = pd.read_csv(filepath, sep=';', decimal=',')
        print(f"Файл {filepath} успешно загружен. Размер: {df.shape}")
        return df
    except Exception as e:
        print(f"Ошибка при загрузке {filepath}: {e}")
        return None

def analyze_clustering():
    df = load_data(FILE_CLUSTERS)
    if df is None: return

    df.columns = [c.replace('|', '_').replace(' ', '_') for c in df.columns]
    
    cluster_col = 'cluster_id' if 'cluster_id' in df.columns else 'Номер_кластера'
    
    feature_candidates = [
        c for c in df.columns 
        if ('Сумма' in c or 'Количество' in c or 'Среднее' in c or 'уникальных' in c or 'Amount' in c)
        and 'DateTime' not in c
    ]
    
    X = df[feature_candidates].select_dtypes(include=[np.number])
    X = X.dropna()
    
    labels = df.loc[X.index, cluster_col]

    print(f"\nИспользуемые признаки для анализа: {X.columns.tolist()}")

    if len(set(labels)) > 1:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        print("\n--- Метрики качества кластеризации ---")
        print(f"Silhouette Score:      {silhouette_score(X_scaled, labels):.4f}")
        print(f"Davies-Bouldin Index:  {davies_bouldin_score(X_scaled, labels):.4f}")
        print(f"Calinski-Harabasz:     {calinski_harabasz_score(X_scaled, labels):.2f}")
    else:
        print("\nВнимание: Найден только 1 кластер или ошибка в данных.")

    print("\n--- Профили кластеров (Средние значения) ---")
    profiles = df.groupby(cluster_col)[X.columns].mean().sort_values(by=X.columns[0])
    profiles['Count_Clients'] = df[cluster_col].value_counts()
    print(profiles)
    
    print("\n--- Сравнение с центроидами (из coordinates.txt) ---")
    df_coords = load_data(FILE_COORDS)
    if df_coords is not None:
        print(df_coords.head())

if __name__ == "__main__":
    analyze_clustering()

Файл clusters.txt успешно загружен. Размер: (4999, 11)

Используемые признаки для анализа: ['transactionAmount_Сумма', 'transactionAmount_Среднее', 'merchantCategoryCode_Кол-во_уникальных']

--- Метрики качества кластеризации ---
Silhouette Score:      0.3666
Davies-Bouldin Index:  0.9569
Calinski-Harabasz:     592.32

--- Профили кластеров (Средние значения) ---
            transactionAmount_Сумма  transactionAmount_Среднее  \
cluster_id                                                       
0                      4.707833e+01                  10.600208   
30                     1.109000e+02                  10.967667   
31                     2.342869e+02                  22.296359   
1                      4.278239e+02                  46.817127   
60                     4.369603e+02                  21.317326   
...                             ...                        ...   
219                    4.070540e+05                 150.113017   
217                    7.220093e+05     